In [1]:
import sys
sys.path.append("../base")

import boilerplate
import alu_assignment

import pyrtl

from pyrtl import *
from assembler import asm, disasm, disasm_pretty

import z3
z3.set_param(proof=True)

In [2]:
from instruction_set import *
from test1 import BLOCKS


reset_working_block()

a = asm(BLOCKS, start_addr=0x5400)    


pc = Register(name='pc', bitwidth=16)
sp = Register(name='sp', bitwidth=16)
r0 = Register(name='r0', bitwidth=16)
r1 = Register(name='r1', bitwidth=16)
mem = MemBlock(name='mem', bitwidth=16, addrwidth=32,
               max_read_ports=30, asynchronous=True)
rom = RomBlock(name='rom', bitwidth=20, addrwidth=20, romdata=a.words,
               pad_with_zeros=True)  # needed for C compilation
out = Output(name='out', bitwidth=4)

[251, 11, 28, 41, 192, 25, 344340, 344251, 28, 59, 344359, 27, 41, 32, 344331, 345015, 344087, 15, 344571, 44, 44, 123, 41, 80, 41, 16, 655371, 41, 16, 139, 344615, 41, 25, 2, 11, 28, 41, 192, 25, 344964, 12, 75, 41, 80, 44, 41, 16, 4091, 28, 41, 13, 25, 27, 41, 32, 344615, 41, 25, 2, 786443, 25, 14, 786443, 25, 14, 28, 41, 32, 25, 345060, 25, 25, 2, 11, 28, 41, 192, 25, 345396, 27, 41, 32, 345239, 25, 25, 2]


In [3]:
def push(val):
    mem[sp] |= val
    sp.next |= sp + 1
    pc.next |= pc + 1

In [4]:
def pop(cnt):
    with cnt == 1:
        with sp > 0:
            r0.next |= mem[sp - 1]
            sp.next |= sp - 1
            pc.next |= pc + 1
    with cnt == 2:
        with sp > 1:
            r1.next |= mem[sp - 1]
            r0.next |= mem[sp - 2]
            sp.next |= sp - 2
            pc.next |= pc + 1
    

In [5]:
def dup(ofs):
    with (sp - ofs) > 0:
        val = mem[sp - ofs - 1]
        push(val)
    with (sp - ofs) <= 0:
        pc.next |= pc + 1

In [6]:
from instruction_set import AOP
from alu_assignment import ALU_function


def alu(op):
    push(ALU_function(r0, r1, op))
    

In [7]:
def load():
    tmp = mem[r0]
    push(tmp)

In [8]:
def store():
    tmp = r0
    mem[r1] |= tmp
    pc.next |= pc + 1

In [9]:
def jmp(addr):
    pc.next |= addr

In [10]:
def jz(addr):
    zero_const = pyrtl.Const(0, bitwidth=16)
    with r0 == zero_const:
        jmp(addr)
    with r0 != zero_const:
        pc.next |= pc + 1

In [11]:
def jnz(addr):
    zero_const = pyrtl.Const(0, bitwidth=16)
    with r0 != zero_const:
        jmp(addr)
    with r0 == zero_const:
        pc.next |= pc + 1

In [12]:
def ret(flag):
    jmp(r0)
    with flag == 1:
        push(r1)

In [13]:
import instruction_set

instr = rom[pc]
out <<= mem[sp]

op = instr[0:4]

with conditional_assignment:
    with (op == PUSH):
        val = instr[4:]
        push(val)
    with (op == POP):
        cnt = instr[4:]
        pop(cnt)
    with (op == DUP):
        ofs = instr[4:]
        dup(ofs)
    with (op == STOR):
        store()
    with (op == LOAD):
        load()
    with (op == JMP):
        addr = instr[4:]
        jmp(addr)
    with (op == JZ):
        addr = instr[4:]
        jz(addr)
    with (op == JNZ):
        addr = instr[4:]
        jnz(addr)
    with (op == RET):
        flag = instr[4:]
        jmp(flag)
    with (op == ALU):
        alu_op = instr[4:]
        alu(alu_op)
    
        

In [14]:
d_sp = Output(name='d_sp', bitwidth=16)
d_pc = Output(name='d_pc', bitwidth=16)
d_r0 = Output(name='d_r0', bitwidth=16)
d_r1 = Output(name='d_r1', bitwidth=16)
d_instr = Output(name='d_instr', bitwidth=16)

d_sp <<= sp
d_pc <<= pc
d_r0 <<= r0
d_r1 <<= r1
d_instr <<= instr

# Debug the lowest 16 memory addresses
d_memaddrs = range(16)
reads = [mem[i] for i in d_memaddrs]
arr = Output(bitwidth=len(reads)*16, name="d_mem")
arr <<= concat_list(reads)

In [15]:
vid_y = Output(name='vid_y', bitwidth=8)
vid_out = Output(name='vid_out', bitwidth=256)

vid_scan = Register(name='vid_scan', bitwidth=5)
vid_scan.next <<= vid_scan + 1
vid_y <<= 18 + vid_scan
vid_out <<= 0xff0f0ff0000

In [16]:
import os
from simulate import CCompiledSimulation

In [17]:
CCompiledSimulation(out_dir="obj")

In [18]:
os.system("make")

gcc -O3 -Iobj -c -o obj/csim.o obj/csim.c
g++ -O3 -Iobj -std=c++11 -DNO_DEBUG_MEM -c -o obj/csim_main.o simulate/csim_main.cpp
g++ -pthread -o bin/csim obj/csim.o obj/csim_main.o


0

In [19]:
# If you don't have GNU make
if 0:
    os.system("gcc -O3 -Iobj -c obj/csim.c")
    os.system("g++ -pthread -Iobj --std=c++11 obj/csim.o simulate/csim_main.cpp -o bin/csim")

In [20]:
os.environ['DEBUG_CPU'] = '1'
os.environ['DEBUG_MEM'] = '1'
os.environ['MAX_CYCLES'] = '100'
os.environ['OUT_DISPLAY'] = '0'
os.system('bin/csim');

0000 0000 0000 0000  000000fb
     0000 0000 0000 0000 0000 0000 0000 0000
     0000 0000 0000 0000 0000 0000 0000 0000
0001 0001 0000 0000  0000000b
     000f 0000 0000 0000 0000 0000 0000 0000
     0000 0000 0000 0000 0000 0000 0000 0000
0002 0002 0000 0000  0000001c
     000f 0000 0000 0000 0000 0000 0000 0000
     0000 0000 0000 0000 0000 0000 0000 0000
0003 0003 0000 0000  00000029
     000f 0000 000f 0000 0000 0000 0000 0000
     0000 0000 0000 0000 0000 0000 0000 0000
0004 0001 0000 000f  000000c0
     000f 0000 000f 0000 0000 0000 0000 0000
     0000 0000 0000 0000 0000 0000 0000 0000
0005 0002 0000 000f  00000019
     000f 0001 000f 0000 0000 0000 0000 0000
     0000 0000 0000 0000 0000 0000 0000 0000
0006 0001 0001 000f  00004114
     000f 0001 000f 0000 0000 0000 0000 0000
     0000 0000 0000 0000 0000 0000 0000 0000
0007 0001 0001 000f  000040bb
     000f 0001 000f 0000 0000 0000 0000 0000
     0000 0000 0000 0000 0000 0000 0000 0000
0008 0002 0001 000f  0000001c
     000f 